# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="../img/reflection_pattern.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: the array to be sorted
    :return: the sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: the first sorted array
    :param right: the second sorted array
    :return: the merged sorted array
    """
    result = []
    i = j = 0

    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    result.extend(left[i:])
    result.extend(right[j:])

    return result
```
Here's an explanation of the code:

The `merge_sort` function takes an array as input and recursively divides it into two halves until each half has a length of 1 or less. Then, it merges the two halves using the `merge` function.

The `merge` function takes two sorted arrays as input and merges them into a single sorted array. It does this by comparing elements from each array and adding the smaller element to the result array. If one array is exhausted, it adds the remaining elements from the other array to the result.

Note that this implementation has a time complexity of O(n log n) and a space complexity of O(n), making it an efficient sorting algorithm for large datasets.

Do you have any feedback or would you like me to revise anything?

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

Overall, your implementation of the Merge Sort algorithm is correct and well-documented. Here are some minor suggestions for improvement:

1. **Docstrings:** Your docstrings are excellent, but you might consider adding a `:rtype:` specification to indicate the return type of each function. For example, `:rtype: list` would indicate that the function returns a list.

2. **Type Hints:** Adding type hints for function parameters and return types can improve code readability and help with static type checking. For example, `def merge_sort(arr: list) -> list:` would indicate that the function takes a list as input and returns a list.

3. **Error Handling:** Your implementation assumes that the input array only contains comparable elements (e.g., integers or strings). However, if the input array contains uncomparable elements (e.g., a mix of integers and strings), your implementation will raise a `TypeError`. You might consider adding error handling to handle such cases.

4. **Code Organization:** Your code is well-organized, but you might consider moving the `merge` function inside the `merge_sort` function as a nested function. This would emphasize that the `merge` function is a helper function for the `merge_sort` function and is not intended to be used independently.

Here is the revised code incorporating these suggestions:

```
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: the array to be sorted
    :return: the sorted array
    :rtype: list
    """
    if len(arr) <= 1:
        return arr

    def merge(left: list, right: list) -> list:
        """
        Merges two sorted arrays into a single sorted array.

        :param left: the first sorted array
        :param right: the second sorted array
        :return: the merged sorted array
        :rtype: list
        """
        result = []
        i = j = 0

        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1

        result.extend(left[i:])
        result.extend(right[j:])

        return result

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)
```

Overall, your implementation is well-written and easy to understand. With these minor suggestions, it's even better!

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the feedback! I'm glad to hear that my implementation is correct and well-documented. Your suggestions are excellent and I've incorporated them into the revised code.

Here's the revised code with the suggested improvements:

```
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: the array to be sorted
    :return: the sorted array
    :rtype: list
    """
    if len(arr) <= 1:
        return arr

    def merge(left: list, right: list) -> list:
        """
        Merges two sorted arrays into a single sorted array.

        :param left: the first sorted array
        :param right: the second sorted array
        :return: the merged sorted array
        :rtype: list
        """
        result = []
        i = j = 0

        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1

        result.extend(left[i:])
        result.extend(right[j:])

        return result

    try:
        mid = len(arr) // 2
        left = arr[:mid]
        right = arr[mid:]

        left = merge_sort(left)
        right = merge_sort(right)

        return merge(left, right)
    except TypeError:
        raise ValueError("The input array must contain comparable elements")

```

I've added type hints for function parameters and return types, as well as `:rtype:` specifications in the docstrings. I've also moved the `merge` function inside the `merge_sort` function as a nested function, which emphasizes its role as a helper function.

Regarding error handling, I've added a `try-except` block to catch `TypeError` exceptions that occur when the input array contains uncomparable elements. In such cases, the function raises a `ValueError` with a more informative error message.

Thank you again for your feedback! I'm committed to writing high-quality Python code, and your suggestions have helped me improve my implementation of the Merge Sort algorithm.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [13]:
from agentic_patterns import ReflectionAgent

In [14]:
agent = ReflectionAgent()

In [16]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [17]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)


STEP 1/10

 

GENERATION

 **Merge Sort Algorithm Implementation in Python**

Merge Sort is a divide-and-conquer algorithm that splits a list of elements into smaller chunks, sorts each chunk, and then merges the sorted chunks into a single, sorted list.

### Code Implementation

```python
def merge_sort(arr):
    """
    Sorts a list of elements using the Merge Sort algorithm.

    Args:
        arr (list): The list of elements to be sorted.

    Returns:
        list: A new list containing the sorted elements.
    """

    # Base case: If the list has 1 or fewer elements, it is already sorted.
    if len(arr) <= 1:
        return arr

    # Split the list into two halves.
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves into a single, sorted list.
    return merge(left_half, right_half)


def merge(left, right):
 

## Final result

In [20]:
display_markdown(final_response, raw=True)

**Code Review**

The updated code includes several improvements, including better error handling and docstrings for test cases. Here are some minor suggestions for further enhancement:

1.  **Input Validation for Array Elements**: While the code checks if the input is a list, it does not validate the elements within the list. Consider adding a check to ensure all elements in the list are integers.

2.  **Code Comments**: While docstrings are present for functions and test cases, consider adding additional code comments to explain the logic and any complex operations.

3.  **Type Hints**: Type hints are used for function parameters, but consider adding type hints for function return values as well.

4.  **Test Case Refactoring**: Some test cases have similar expected behaviors. Consider refactoring these test cases to reduce duplication.

Here is the refactored code with the suggestions applied:

```python
from typing import List

def merge_lists(left: List[int], right: List[int]) -> List[int]:
    """
    Merges two sorted lists into a single, sorted list.

    Args:
        left (List[int]): The first sorted list.
        right (List[int]): The second sorted list.

    Returns:
        List[int]: A new list containing the merged, sorted elements.

    Time Complexity:
        O(n)

    Space Complexity:
        O(n)
    """

    # Merge the two lists with a temporary list
    merged = []
    left_index = 0
    right_index = 0

    # Merge until one list is exhausted
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Merge in remaining elements, if any
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


def validate_input(arr: List[int]) -> bool:
    """
    Validates the input for merge_sort and handles potential errors.

    Args:
        arr (List[int]): The list of elements to be sorted.

    Returns:
        bool: True if the input is valid, False otherwise.
    """

    try:
        # Input validation for Merge Sort
        if not isinstance(arr, list):
            raise ValueError("Input must be a list")
        if not all(isinstance(x, int) for x in arr):
            raise ValueError("All elements in the list must be integers")
        return True
    except ValueError as e:
        print(f"Error: {e}")
        return False


def merge_sort(arr: List[int]) -> List[int]:
    """
    Sorts a list of elements using the Merge Sort algorithm.

    Args:
        arr (List[int]): The list of elements to be sorted.

    Returns:
        List[int]: A new list containing the sorted elements.

    Time Complexity:
        O(n log n)

    Space Complexity:
        O(n)
    """

    # Arrays of length 1 are already sorted
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort both halves
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves
    return merge_lists(left_half, right_half)


# Example usage:
if __name__ == "__main__":
    # Test cases
    test_cases = [
        {
            "name": "Testing merge_sort with an empty list:",
            "arr": [],
            "expected": [],
            "doc": """
            Verifies the correctness of merge_sort with an empty list.
            This test case checks if the function correctly handles an empty list
            and returns an empty list.
            """
        },
        {
            "name": "Testing merge_sort with a single-element list:",
            "arr": [5],
            "expected": [5],
            "doc": """
            Verifies the correctness of merge_sort with a single-element list.
            This test case checks if the function correctly handles a list with a single element
            and returns the same list.
            """
        },
        {
            "name": "Testing merge_sort with a list containing duplicate elements:",
            "arr": [4, 2, 6, 2, 1, 4],
            "expected": [1, 2, 2, 4, 4, 6],
            "doc": """
            Verifies the correctness of merge_sort with a list containing duplicate elements.
            This test case checks if the function correctly handles a list with duplicate elements
            and returns a sorted list with the same elements.
            """
        },
        {
            "name": "Testing merge_sort with a list containing negative numbers:",
            "arr": [-10, 2, -5, 8, 0, 4],
            "expected": [-10, -5, 0, 2, 4, 8],
            "doc": """
            Verifies the correctness of merge_sort with a list containing negative numbers.
            This test case checks if the function correctly handles a list with negative numbers
            and returns a sorted list with the same elements in the correct order.
            """
        },
        {
            "name": "Testing merge_sort with a large list:",
            "arr": [64, 34, 25, 12, 22, 11, 90],
            "expected": [11, 12, 22, 25, 34, 64, 90],
            "doc": """
            Verifies the correctness of merge_sort with a large list.
            This test case checks if the function correctly handles a large list
            and returns a sorted list with all elements in the correct order.
            """
        }
    ]

    for test_case in test_cases:
        print(test_case["name"])
        print(test_case["doc"])

        if not validate_input(test_case["arr"]):
            print("Input validation failed")
            continue

        sorted_arr = merge_sort(test_case["arr"])

        print("Original array:", test_case["arr"])
        print("Sorted array:", sorted_arr)
        print("Expected array:", test_case["expected"])

        try:
            assert sorted_arr == test_case["expected"], f"Expected {test_case['expected']} but got {sorted_arr}"
        except AssertionError as e:
            print("Assertion Error:", str(e))
        print()
```

**Changes**:

1.  **Input Validation for Array Elements**: The `validate_input` function has been updated to check if all elements in the list are integers.
2.  **Code Comments**: Additional code comments have been added to explain the logic and any complex operations.
3.  **Type Hints**: Type hints have been added for function return values.
4.  **Test Case Refactoring**: Similar test cases have been refactored to reduce duplication.